In [1]:
import pyautogui
import shutil
import time
from PIL import ImageGrab
from PIL import Image
import os
import keyboard
import pyperclip
import wx
import threading

flag = 0
def stop():
    keyboard.wait('f2')
    flag=1
    
t=threading.Thread(target=stop)
# t.start()

image_path = 'D:/AutoClickPicture/'
cache = ''
try:
    os.mkdir(image_path)
except:
    pass
def click_image(image, confidence=0.95, lag_x=0,lag_y=0,wait=0.2, drag=None, double=False,loop=False):
    try:
        while True:
            # 获取屏幕分辨率
            screen_width, screen_height = pyautogui.size()

            # 全屏截图
            screen_image = ImageGrab.grab()

            # 在屏幕截图中查找匹配
            location = pyautogui.locate(image, screen_image, confidence=confidence)
            if location:
                time.sleep(wait)
                location = pyautogui.locate(image, screen_image, confidence=confidence)
                # 匹配成功，返回位置信息
                x, y, a, b = location
                if drag==None:
                    if double==False:
                        pyautogui.click(x+a//2+lag_x, y+b//2+lag_y)
                    else:
                        pyautogui.doubleClick(x+a//2+lag_x, y+b//2+lag_y)
                else:
                    pyautogui.moveTo(x+a//2+lag_x, y+b//2+lag_y)
                    pyautogui.mouseDown()
                    pyautogui.moveRel(drag[0],drag[1],0.2)
                    pyautogui.mouseUp()
                return True
            else:
                # 匹配失败，返回None
                if loop==False or flag==1:return False
                return
                time.sleep(10)    
    except Exception as e:
        print(e)

In [6]:
class btnFrame(wx.Frame):
    def __init__( self, parent ):
        self.flag = -1
        self.temp=''
        self.pic_group =[]
        if cache !='':self.m_textCtrl1.SetValue(cache)
        wx.Frame.__init__ ( self, parent, id = wx.ID_ANY, title = wx.EmptyString, pos = wx.DefaultPosition, size = wx.Size( 500,300 ), style = wx.DEFAULT_FRAME_STYLE|wx.TAB_TRAVERSAL )

        self.SetSizeHints( wx.DefaultSize, wx.DefaultSize )

        bSizer1 = wx.BoxSizer( wx.HORIZONTAL )

        bSizer2 = wx.BoxSizer( wx.VERTICAL )

        self.m_staticText1 = wx.StaticText( self, wx.ID_ANY, u"添加图片", wx.DefaultPosition, wx.DefaultSize, 0 )
        self.m_staticText1.Wrap( -1 )

        bSizer2.Add( self.m_staticText1, 0, wx.ALL, 5 )

        self.m_button6 = wx.Button( self, wx.ID_ANY, u"单击", wx.DefaultPosition, wx.DefaultSize, 0 )
        bSizer2.Add( self.m_button6, 0, wx.ALL, 5 )
        self.m_button6.Bind(wx.EVT_BUTTON, self.On_click_btn)

        self.m_button7 = wx.Button( self, wx.ID_ANY, u"双击", wx.DefaultPosition, wx.DefaultSize, 0 )
        bSizer2.Add( self.m_button7, 0, wx.ALL, 5 )
        self.m_button7.Bind(wx.EVT_BUTTON, self.On_click_btn)

        self.m_button16 = wx.Button( self, wx.ID_ANY, u"单击且暂停", wx.DefaultPosition, wx.DefaultSize, 0 )
        bSizer2.Add( self.m_button16, 0, wx.ALL, 5 )
        self.m_button16.Bind(wx.EVT_BUTTON, self.On_click_btn)

        self.m_staticText3 = wx.StaticText( self, wx.ID_ANY, u"调试", wx.DefaultPosition, wx.DefaultSize, 0 )
        self.m_staticText3.Wrap( -1 )

        bSizer2.Add( self.m_staticText3, 0, wx.ALL, 5 )

        self.m_button8 = wx.Button( self, wx.ID_ANY, u"单步调试", wx.DefaultPosition, wx.DefaultSize, 0 )
        bSizer2.Add( self.m_button8, 0, wx.ALL, 5 )
        self.m_button8.Bind(wx.EVT_BUTTON, self.On_debug_btn)

        self.m_button13 = wx.Button( self, wx.ID_ANY, u"重置调试", wx.DefaultPosition, wx.DefaultSize, 0 )
        bSizer2.Add( self.m_button13, 0, wx.ALL, 5 )
        self.m_button13.Bind(wx.EVT_BUTTON, self.On_reset_btn)


        bSizer1.Add( bSizer2, 1, wx.EXPAND, 5 )

        bSizer4 = wx.BoxSizer( wx.VERTICAL )

        self.m_staticText4 = wx.StaticText( self, wx.ID_ANY, u"文件夹名称", wx.DefaultPosition, wx.DefaultSize, 0 )
        self.m_staticText4.Wrap( -1 )

        bSizer4.Add( self.m_staticText4, 0, wx.ALL, 5 )

        self.m_textCtrl1 = wx.TextCtrl( self, wx.ID_ANY, wx.EmptyString, wx.DefaultPosition, wx.DefaultSize, 0 )
        bSizer4.Add( self.m_textCtrl1, 0, wx.ALL, 5 )

        self.m_button9 = wx.Button( self, wx.ID_ANY, u"保存", wx.DefaultPosition, wx.DefaultSize, 0 )
        bSizer4.Add( self.m_button9, 0, wx.ALL, 5 )
        self.m_button9.Bind(wx.EVT_BUTTON, self.On_save_btn)

        self.m_button10 = wx.Button( self, wx.ID_ANY, u"加载", wx.DefaultPosition, wx.DefaultSize, 0 )
        bSizer4.Add( self.m_button10, 0, wx.ALL, 5 )
        self.m_button10.Bind(wx.EVT_BUTTON, self.On_load_btn)

        self.m_button14 = wx.Button( self, wx.ID_ANY, u"执行", wx.DefaultPosition, wx.DefaultSize, 0 )
        bSizer4.Add( self.m_button14, 0, wx.ALL, 5 )
        self.m_button14.Bind(wx.EVT_BUTTON, self.On_start_btn)

        self.m_button15 = wx.Button( self, wx.ID_ANY, u"停止", wx.DefaultPosition, wx.DefaultSize, 0 )
        bSizer4.Add( self.m_button15, 0, wx.ALL, 5 )
        self.m_button15.Bind(wx.EVT_BUTTON, self.On_stop_btn)


        bSizer1.Add( bSizer4, 1, wx.EXPAND, 5 )

        bSizer8 = wx.BoxSizer( wx.VERTICAL )
        
        self.m_staticText5 = wx.StaticText( self, wx.ID_ANY, u"运行信息", wx.DefaultPosition, wx.DefaultSize, 0 )
        self.m_staticText5.Wrap( -1 )

        bSizer8.Add( self.m_staticText5, 0, wx.ALL, 5 )

        self.m_textCtrl2 = wx.TextCtrl( self, wx.ID_ANY, wx.EmptyString, wx.DefaultPosition, wx.DefaultSize, 0 )
        bSizer8.Add( self.m_textCtrl2, 0, wx.ALL, 5 )
        
        self.m_button12 = wx.Button( self, wx.ID_ANY, u"始终置顶", wx.DefaultPosition, wx.DefaultSize, 0 )
        bSizer8.Add( self.m_button12, 0, wx.ALL, 5 )
        self.m_button12.Bind(wx.EVT_BUTTON, self.On_toggle_btn)


        bSizer1.Add( bSizer8, 1, wx.EXPAND, 5 )


        self.SetSizer( bSizer1 )
        self.Layout()

        self.Centre( wx.BOTH )
        self.Show()
        self.SetWindowStyle(wx.DEFAULT_FRAME_STYLE | wx.STAY_ON_TOP)
        
    def On_toggle_btn(self, event):
        if self.GetWindowStyle() & wx.STAY_ON_TOP:
            self.SetWindowStyle(wx.DEFAULT_FRAME_STYLE)
            self.m_textCtrl2.SetValue('取消置顶')
        else:
            self.SetWindowStyle(wx.DEFAULT_FRAME_STYLE | wx.STAY_ON_TOP)
            self.m_textCtrl2.SetValue('已置顶')
            
    def On_click_btn(self, event):
        self.flag +=1
        self.pic_group.insert(self.flag,ImageGrab.grabclipboard())
        if click_image(self.pic_group[self.flag]):
            self.m_textCtrl2.SetValue('点击成功')
        else:
            self.m_textCtrl2.SetValue('点击失败')
    
    def On_debug_btn(self, event):
        x,y=pyautogui.position()
        if self.flag==len(self.pic_group)-1:return
        self.flag+=1
        if click_image(self.pic_group[self.flag]):
            self.m_textCtrl2.SetValue('点击成功')
        else:
            self.m_textCtrl2.SetValue('点击失败')  
        pyautogui.moveTo(x,y)
        
    def On_reset_btn(self, event):
        self.flag=-1
    
    def On_save_btn(self, event):
        global cache
        self.temp = self.m_textCtrl1.GetValue()
        cache = self.temp
        try:
            os.mkdir(image_path+f'{self.temp}/')
        except:
            shutil.rmtree(image_path+f'{self.temp}/')
            os.mkdir(image_path+f'{self.temp}/')
        for i in range(0,len(self.pic_group)):
            self.pic_group[i].save(image_path+f'{self.temp}/'+f'{i}.png')
        
    def On_load_btn(self, event):
        global cache
        cache = self.temp
        self.temp = self.m_textCtrl1.GetValue()
        self.pic_group=[]
        self.flag=-1
        for i in os.listdir(image_path+f'{self.temp}/'):
            image = Image.open(image_path+f'{self.temp}/{i}')
            self.pic_group.append(image)
            
    def On_start_btn(self, event):
        self.m_textCtrl2.SetValue('')
        l=len(self.pic_group)
        for i in range(l):
            click_image(self.pic_group[i],loop=True)
            self.m_textCtrl2.SetValue(f'{i}执行完成')
        self.m_textCtrl2.SetValue('执行完成')
    
    def On_stop_btn(self, event):
        time.sleep(10)
        l=len(self.pic_group)
        for i in range(l):
            click_image(self.pic_group[i],loop=True)
            self.m_textCtrl2.SetValue(f'{i}执行完成')
        self.m_textCtrl2.SetValue('执行完成')
        
try:
    del app
except:
    pass      
app = wx.App()
btn_group = {'toggle_btn':'始终置顶','add_btn':'添加',\
                     'debug_btn':'调试','reset_btn':'重置','save_btn':'保存','load_btn':'加载','start_btn':'执行'}
frame = btnFrame(None)
app.MainLoop()

0